In [21]:
import math

from autograd import grad, hessian, jacobian, hessian_vector_product

import autograd.numpy as np
import autograd.numpy.random as npr

import copy
import scipy
from scipy import optimize
from scipy import stats

In [81]:
K = 500
mat = np.full((K, K), 0.1) + np.eye(K)
loc = np.array([float(k) / 7. for k in range(K)])
def ObjPar(par, mat, loc):
    diff = par - loc
    return np.dot(diff, np.matmul(mat, diff))

x = loc + 0.1 * np.random.rand(K)

ObjPar(x, mat, loc)
Obj = lambda x: ObjPar(x, mat, loc)
ObjHess = hessian(Obj)
ObjHessVecProd = hessian_vector_product(Obj)

print Obj(x)

68.58036019


In [82]:
from scipy.sparse.linalg import LinearOperator
import timeit

x0 = loc
ObjHessVecProdLO = LinearOperator((K, K), lambda par: ObjHessVecProd(x0, par))
ObjHessVecProdLO(x)

cg_res, info = scipy.sparse.linalg.cg(ObjHessVecProdLO, x)
ls_res = np.linalg.solve(ObjHess(x0), x)

time_num = 10
cg_time = timeit.timeit(lambda: scipy.sparse.linalg.cg(ObjHessVecProdLO, x), number=time_num) / time_num
ls_time = timeit.timeit(lambda: np.linalg.solve(ObjHess(x0), x), number=time_num) / time_num

print cg_time
print ls_time

print max(np.abs(cg_res - ls_res))

0.00450010299683
0.434067296982
1.13686837722e-13
